In [28]:
from functools import reduce
import pandas as pd
import glob
import os

# Caminho da pasta com os arquivos CSV
pasta = './'  # ajuste se necessário

# Lista todos os arquivos CSV que começam com 'investing'
arquivos_csv = glob.glob(os.path.join(pasta, '*.csv'))

# Lê e concatena todos os arquivos
dfs = [pd.read_csv(f) for f in arquivos_csv]

# Converte a coluna de datas (ajuste o nome da coluna se necessário)

# Renomeia as colunas de fechamento e mínima em cada DataFrame e faz o merge por 'Data'
dfs_renomeados = []
nomes = ['DIVO', 'HASH', 'IVV']

for df_atual, nome in zip(dfs, nomes):
    temp = df_atual[['Data', 'Último', 'Mínima']].copy()
    temp['Data'] = pd.to_datetime(temp['Data'], dayfirst=True, errors='coerce', format='%d.%m.%Y')
    temp = temp.rename(columns={
        'Último': f'c_{nome}',
        'Mínima': f'l_{nome}'
    })
    dfs_renomeados.append(temp)

# Merge dos três DataFrames pela coluna 'Data'
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Data', how='outer'), dfs_renomeados)

# Ordena por data crescente
df = df_merged.dropna().sort_values('Data', ascending=True).reset_index(drop=True)

# Converte as colunas de preços para float (substitui vírgula por ponto)
for nome in nomes:
    df[f'c_{nome}'] = df[f'c_{nome}'].str.replace(',', '.').astype(float)
    df[f'l_{nome}'] = df[f'l_{nome}'].str.replace(',', '.').astype(float)

for nome in nomes:
    df[f'r_{nome}_pos_tax'] = (df[f'c_{nome}'] / df[f'c_{nome}'].shift(1) - 1) * 0.85  # considerando 15% de imposto
    df[f'ra_{nome}_buyhold'] = df[f'r_{nome}_pos_tax'].cumsum()
    df[f'r_{nome}_maxdown'] = (df[f'l_{nome}'] / df[f'c_{nome}'].shift(1) - 1)

df = df.dropna().sort_values('Data', ascending=True)

# Salva em Excel
df.to_excel('análise.xlsx', index=False)
df

,Data,c_DIVO,l_DIVO,c_HASH,l_HASH,c_IVV,l_IVV,r_DIVO_pos_tax,ra_DIVO_buyhold,r_DIVO_maxdown,r_HASH_pos_tax,ra_HASH_buyhold,r_HASH_maxdown,r_IVV_pos_tax,ra_IVV_buyhold,r_IVV_maxdown
1,2021-06-07,74.41,74.00,34.90,34.62,231.46,230.01,0.002750,0.002750,-0.002292,-0.013188,-0.013188,-0.023413,-0.000917,-0.000917,-0.007337
2,2021-06-08,73.85,73.83,32.40,30.16,231.47,229.64,-0.006397,-0.003647,-0.007795,-0.060888,-0.074076,-0.135817,0.000037,-0.000880,-0.007863
3,2021-06-09,73.67,73.67,34.60,33.13,232.00,231.12,-0.002072,-0.005718,-0.002437,0.057716,-0.016360,0.022531,0.001946,0.001066,-0.001512
4,2021-06-10,73.81,73.64,34.45,34.18,234.10,231.70,0.001615,-0.004103,-0.000407,-0.003685,-0.020045,-0.012139,0.007694,0.008760,-0.001293
5,2021-06-11,73.99,72.50,34.72,34.31,237.40,234.25,0.002073,-0.002030,-0.017748,0.006662,-0.013383,-0.004064,0.011982,0.020742,0.000641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,2025-09-24,107.57,107.36,86.41,85.74,396.50,395.03,-0.000316,0.362791,-0.002323,0.014812,1.297788,0.009537,0.003422,0.517885,0.000304
1081,2025-09-25,106.45,106.30,83.50,82.62,397.79,393.14,-0.008850,0.353941,-0.011806,-0.028625,1.269163,-0.043861,0.002765,0.520650,-0.008474
1082,2025-09-26,106.76,106.35,83.73,82.86,398.25,396.00,0.002475,0.356416,-0.000939,0.002341,1.271504,-0.007665,0.000983,0.521633,-0.004500
1083,2025-09-29,107.03,107.03,86.91,84.93,398.00,396.36,0.002150,0.358566,0.002529,0.032282,1.303787,0.014332,-0.000534,0.521099,-0.004746
